## Connecting Data to Mysql

In [42]:
from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override() 

In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error as mse
import numpy as np 
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.api import SimpleExpSmoothing, Holt
import matplotlib.pyplot as plt

In [18]:
def stock_df(tickers, start_dt, end_dt):
    start_date = pd.to_datetime(start_dt)
    end_date = pd.to_datetime(end_dt)
    
    period_limit = pd.DateOffset(months=1)
    stock_data = pd.DataFrame()
    
    while start_date <= end_date:
        # Calculate the end date for this period
        period_end = min(start_date + period_limit, end_date)

        # Download stock data for this period
        data = yf.download(tickers, start=start_date, end=period_end)

        # Concatenate the data with the existing DataFrame
        stock_data = pd.concat([stock_data, data])

        # Move to the next period
        start_date = period_end + pd.Timedelta(days=1)
    
    return stock_data

In [19]:
tcs_df=stock_df("TCS.NS","2020-01-01","2024-02-14")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [20]:
tcs_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-01,2168.000000,2183.899902,2154.000000,2167.600098,1999.318604,1354908
2020-01-02,2179.949951,2179.949951,2149.199951,2157.649902,1990.141235,2380752
2020-01-03,2164.000000,2223.000000,2164.000000,2200.649902,2029.802612,4655761
2020-01-06,2205.000000,2225.949951,2187.899902,2200.449951,2029.619141,3023209
2020-01-07,2200.500000,2214.649902,2183.800049,2205.850098,2034.599609,2429317


## Data Storage:

#### Using MySql database to store data 


In [21]:
import pandas as pd
from sqlalchemy import create_engine

# Define the MySQL database connection parameters
username = 'root'
password = '0808'
host = 'localhost'  # or '127.0.0.1' if MySQL is running locally
port = '3306'  # MySQL default port
database = 'stock_database'

# Create a connection string
connection_string = f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}'

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

# Export DataFrame to MySQL
tcs_df.to_sql('stock_table', engine, index=False, if_exists='replace')

print("DataFrame exported to MySQL successfully.")


DataFrame exported to MySQL successfully.


In [22]:
# SQL query to retrieve data from MySQL table into a DataFrame
query = "SELECT * FROM stock_table"

# Execute the query and load the results into a DataFrame
df_from_mysql = pd.read_sql(query, engine)

print(df_from_mysql.head())


          Open         High          Low        Close    Adj Close   Volume
0  2168.000000  2183.899902  2154.000000  2167.600098  1999.318604  1354908
1  2179.949951  2179.949951  2149.199951  2157.649902  1990.141235  2380752
2  2164.000000  2223.000000  2164.000000  2200.649902  2029.802612  4655761
3  2205.000000  2225.949951  2187.899902  2200.449951  2029.619141  3023209
4  2200.500000  2214.649902  2183.800049  2205.850098  2034.599609  2429317


## Partition data 

In [29]:
pip install pandas pyarrow

     ---------------------------------------- 24.8/24.8 MB 1.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [30]:
def partition_stock_data(stock_data):
    stock_data['Year'] = stock_data.index.year
    stock_data['Month'] = stock_data.index.month
    return stock_data

In [31]:
def save_to_parquet(data, filename):
    data.to_parquet(filename, engine='pyarrow')

In [32]:
# Partition stock data by year and month
partitioned_data = partition_stock_data(tcs_df)

In [36]:
tcs_df_parquet = 'partitioned_stock_data.parquet'

In [37]:
 # Save partitioned data to Parquet format
save_to_parquet(partitioned_data, tcs_df_parquet)

In [38]:
# Load partitioned data from Parquet format
def load_partitioned_data(filename):
    return pd.read_parquet(filename)

In [39]:
input_filename = 'partitioned_stock_data.parquet'
    
# Load partitioned data from Parquet format
partitioned_data = load_partitioned_data(input_filename)
    
# Display the loaded data
print(partitioned_data.head())

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2020-01-01  2168.000000  2183.899902  2154.000000  2167.600098  1999.318604   
2020-01-02  2179.949951  2179.949951  2149.199951  2157.649902  1990.141235   
2020-01-03  2164.000000  2223.000000  2164.000000  2200.649902  2029.802612   
2020-01-06  2205.000000  2225.949951  2187.899902  2200.449951  2029.619141   
2020-01-07  2200.500000  2214.649902  2183.800049  2205.850098  2034.599609   

             Volume  Year  Month  
Date                              
2020-01-01  1354908  2020      1  
2020-01-02  2380752  2020      1  
2020-01-03  4655761  2020      1  
2020-01-06  3023209  2020      1  
2020-01-07  2429317  2020      1  
